- In Theory we use log return to test lots of theroies, for example: CAPM, APT, Statistical Arbitrage...etc.  However, In real word, we have to buy the number of share and tried to achive desire weight. 

- Therefore, the following investment back testing will focrus on 

    -1.) Single Asset: Number of Share, Total Investment, Investment P&L.

    -2.) Multi Asset: Multi asset backt testing without rebalance.

    -3.) Monthly Rebalance: Profit and Loss from Transaction Date, Rebalance wight weight signal systematically


In [1]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

# Single Asset Back-Testing

In [2]:
def daily_investment_value(shares,price):
    return shares * price 


def single_asset_back_testing(price_df,initial_investment,investment_weight,price_column_name):
    """
    """
    investment_value = initial_investment * investment_weight

    price_df['Asset Retrun'] = price_df[[str(price_column_name)]].pct_change()
    price_df['# Share'] = int(investment_value/price_df.iloc[0][str(price_column_name)])

    price_df['Investment Value'] = price_df.apply(lambda x: daily_investment_value(shares=x['# Share'], price=x[str(price_column_name)]),axis=1)
    price_df['Remain Cash'] = investment_value - price_df['# Share'].iloc[0] * price_df[price_column_name].iloc[0]
    price_df['PnL'] = price_df['Investment Value'].diff()
    price_df['To Date. Return'] = (price_df['Investment Value'] / price_df.iloc[0]['Investment Value'])

    return price_df

In [3]:
price_df = yf.download("TSLA", start= dt.datetime(2022,5,1), end = dt.datetime(2022,10,1),interval='1mo').dropna()[['Close']]


initial_investment = 1300
investment_weight = 1
price_column_name = 'Close'

single_asset_back_testing(price_df,initial_investment,investment_weight,price_column_name)

[*********************100%***********************]  1 of 1 completed


,Close,Asset Retrun,# Share,Investment Value,Remain Cash,PnL,To Date. Return
Date,,,,,,,
2022-05-01,252.753326,NaN,5,1263.766632,36.233368,NaN,1.000000
2022-06-01,224.473328,-0.111888,5,1122.366638,36.233368,-141.399994,0.888112
2022-07-01,297.149994,0.323765,5,1485.749969,36.233368,363.383331,1.175652
2022-08-01,275.609985,-0.072489,5,1378.049927,36.233368,-107.700043,1.090431
2022-09-01,265.250000,-0.037589,5,1326.250000,36.233368,-51.799927,1.049442


#  Multi Assets Back-Testing

In [4]:
# Portfolio : 10 Assets
start_date = '2022-10-27'
end_date   = '2022-11-04'

# yfinance likes the tickers formatted as a list
tickers   = ["MSFT","AAPL","AMZN"]
ticks     = yf.Tickers(tickers)
Stock_DF  = ticks.history(start=start_date, end=end_date).Close.dropna()
Stock_DF

[*********************100%***********************]  3 of 3 completed


,AAPL,AMZN,MSFT
Date,,,
2022-10-26,149.350006,115.660004,231.320007
2022-10-27,144.800003,110.959999,226.750000
2022-10-28,155.740005,103.410004,235.869995
2022-10-31,153.339996,102.440002,232.130005
2022-11-01,150.649994,96.790001,228.169998
2022-11-02,145.029999,92.120003,220.100006
2022-11-03,138.880005,89.300003,214.250000


In [5]:
investment_weights = [ 0.5,0.5,0]
initial_investment = 1300
price_column_name = tickers
price_df = Stock_DF

def multi_asset_back_testing(price_df,initial_investment,investment_weights,price_column_name):
    """
    """
    price_df = price_df.copy()
    investment_values = [ weight * initial_investment for weight in investment_weights ]

    price_df[[str(tick) + " return" for tick in tickers]] = price_df[[str(tick) for tick in tickers]].pct_change()
    price_df[[str(tick) + " #Share" for tick in tickers]]= [ int(invesment/price_df[tick].iloc[0]) for (invesment,tick) in zip(investment_values,tickers)]

    for tick in tickers:        
       price_df[str(tick) + " Investment Value"] = price_df.apply(lambda x: daily_investment_value(shares=x[str(tick) + " #Share" ], price=x[str(tick)]),axis=1)

    price_df['Total Investment Value'] = price_df[[str(tick) + " Investment Value" for tick in tickers]].sum(axis=1)
    price_df['Investment PnL'] = price_df['Total Investment Value'] - price_df['Total Investment Value'].iloc[0]
    price_df['Remain Cash'] = initial_investment - price_df['Total Investment Value'].iloc[0]
    price_df['Total Value'] = price_df['Total Investment Value'] + price_df['Remain Cash']

    price_df['Investment To Date. Return(%)'] = ((price_df['Total Investment Value'] / price_df.iloc[0]['Total Investment Value']) -1) *100


    return price_df


multi_asset_back_testing(price_df,initial_investment,investment_weights,price_column_name)

,AAPL,AMZN,MSFT,MSFT return,AAPL return,AMZN return,MSFT #Share,AAPL #Share,AMZN #Share,MSFT Investment Value,AAPL Investment Value,AMZN Investment Value,Total Investment Value,Investment PnL,Remain Cash,Total Value,Investment To Date. Return(%)
Date,,,,,,,,,,,,,,,,,
2022-10-26,149.350006,115.660004,231.320007,NaN,NaN,NaN,2,4,0,462.640015,597.400024,0.0,1060.040039,0.000000,239.959961,1300.000000,0.000000
2022-10-27,144.800003,110.959999,226.750000,-0.019756,-0.030465,-0.040636,2,4,0,453.500000,579.200012,0.0,1032.700012,-27.340027,239.959961,1272.659973,-2.579150
2022-10-28,155.740005,103.410004,235.869995,0.040220,0.075553,-0.068042,2,4,0,471.739990,622.960022,0.0,1094.700012,34.659973,239.959961,1334.659973,3.269685
2022-10-31,153.339996,102.440002,232.130005,-0.015856,-0.015410,-0.009380,2,4,0,464.260010,613.359985,0.0,1077.619995,17.579956,239.959961,1317.579956,1.658424
2022-11-01,150.649994,96.790001,228.169998,-0.017059,-0.017543,-0.055154,2,4,0,456.339996,602.599976,0.0,1058.939972,-1.100067,239.959961,1298.899933,-0.103776
2022-11-02,145.029999,92.120003,220.100006,-0.035368,-0.037305,-0.048249,2,4,0,440.200012,580.119995,0.0,1020.320007,-39.720032,239.959961,1260.279968,-3.747031
2022-11-03,138.880005,89.300003,214.250000,-0.026579,-0.042405,-0.030612,2,4,0,428.500000,555.520020,0.0,984.020020,-76.020020,239.959961,1223.979980,-7.171429


# Monthly Rebalance Problem

    Monthly Rebalance Problem:

    - Profit and Loss from Transaction Date (10/31 - 11/01)
    - Rebalance wight weight signal systematically --> Weight signal dataframe
    - System the backtesting function with input: Price DataFrame, initial investment value, portfolio weight signal per month

In [6]:
# Portfolio : 10 Assets
start_date = '2022-10-27'
end_date   = '2022-11-04'

# yfinance likes the tickers formatted as a list
tickers   = ["MSFT","AAPL","AMZN"]
ticks     = yf.Tickers(tickers)
Stock_DF  = ticks.history(start=start_date, end=end_date).Close.dropna()
Stock_DF

[*********************100%***********************]  3 of 3 completed


,AAPL,AMZN,MSFT
Date,,,
2022-10-26,149.350006,115.660004,231.320007
2022-10-27,144.800003,110.959999,226.750000
2022-10-28,155.740005,103.410004,235.869995
2022-10-31,153.339996,102.440002,232.130005
2022-11-01,150.649994,96.790001,228.169998
2022-11-02,145.029999,92.120003,220.100006
2022-11-03,138.880005,89.300003,214.250000


- Rebalanced Signal [ Initial Investment, Portfolio Wieght Per Month, Price Column Nmae]

In [7]:
investment_weight_1 = [0.5,0.5,0]
investment_weight_2 = [0.1,0.2,0.7]
investment_weights = []
investment_weights.append(investment_weight_1)
investment_weights.append(investment_weight_2)
initial_investment = 1300
price_column_name = Stock_DF.columns

print("Initial Investment: ",initial_investment)
print("Trading Asset: ",price_column_name)

Initial Investment:  1300
Trading Asset:  Index(['AAPL', 'AMZN', 'MSFT'], dtype='object')


- I believe other way to improve this code is that we could set a key coulmn for rebalance, and groupby the rebalance_key columns

In [8]:
def Monthly_Rebalance(Stock_DF,price_column_name,investment_weights,initial_investment):
    """
    Construct { Investment PnL, Remain Cash, Total Value, Investment To Date. Return(%), Investment PnL Change }
    But, we still have to adjust the Profit Transaction Date (ex. 10/31 - 11/01) with another function.
    """
    back_testing_df = pd.DataFrame()
    tickers = price_column_name 
    
    for i,(name,df) in enumerate(Stock_DF.to_period('M').groupby('Date')):
        
        investment_weight = investment_weights[i]
        period_back_testing_df = multi_asset_back_testing(df,initial_investment,investment_weight,price_column_name)

        back_testing_df = pd.concat([back_testing_df, period_back_testing_df],axis=0)
        back_testing_df[[str(tick) + " return" for tick in tickers]] = back_testing_df[[str(tick) for tick in tickers]].pct_change()
        back_testing_df['Investment PnL'] = back_testing_df['Total Investment Value'] - back_testing_df['Total Investment Value'].iloc[0]
        back_testing_df['Investment PnL Change'] = back_testing_df['Investment PnL'].diff().shift(-1)
        initial_investment = back_testing_df['Total Investment Value'].iloc[-1] + back_testing_df['Remain Cash'].iloc[-1]

    back_testing_df.index = Stock_DF.index
    
    return back_testing_df

def Adjust_Backtesting_Result(back_testing_df,price_column_name,investment_weights,initial_investment):
    """
    # Construct { Investment PnL, Remain Cash, Total Value, Investment To Date. Return(%), Investment PnL Change }
    """
    all_back_testing_df = pd.DataFrame()
    

    for i,(name,df) in enumerate(back_testing_df.to_period('M').groupby('Date')):

        tickers = price_column_name 
        price_df = df 
        investment_weight = investment_weights[i]

        # period backtesting with share and index investment value
        period_back_testing_df = multi_asset_back_testing(price_df,initial_investment,investment_weight,price_column_name)
        all_back_testing_df = pd.concat([all_back_testing_df, period_back_testing_df],axis=0)

        # Construct { Investment PnL, Remain Cash, Total Value, Investment To Date. Return(%), Investment PnL Change }
        all_back_testing_df[[str(tick) + " return" for tick in tickers]] = all_back_testing_df[[str(tick) for tick in tickers]].pct_change()
        all_back_testing_df['Investment PnL'] = all_back_testing_df['Total Investment Value'] - all_back_testing_df['Total Investment Value'].iloc[0]
        all_back_testing_df['Investment PnL Change'] = all_back_testing_df['Investment PnL'].diff().shift(-1)
        all_back_testing_df['Investment To Date. Return(%)'] = all_back_testing_df['Total Investment Value'] / all_back_testing_df['Total Investment Value'].iloc[0]

        # reset initial investment value for evey month periods
        initial_investment = period_back_testing_df['Total Investment Value'].iloc[-1] + period_back_testing_df['Remain Cash'].iloc[-1] - period_back_testing_df['Investment PnL Change'].iloc[-1]

    # Split the backtesting result to 3 main table(asset price and return, asset investment share and value, asset backtesting result)
    all_back_testing_df.index = back_testing_df.index
    investment_asset_columns = list([str(tick) for tick in tickers])
    investment_asset_columns.extend([str(tick) + " return" for tick in tickers])
    investment_asset_share =  list([str(tick) + " #Share" for tick in tickers])
    investment_asset_share.extend([str(tick) + " Investment Value" for tick in tickers])
    investment_asset_share.extend(['Total Investment Value'])
    investment_asset_share.extend(['Remain Cash'])
    investment_asset_share.extend(['Total Value'])

    investment_asset_result =  all_back_testing_df[investment_asset_columns]
    investment_share_value = all_back_testing_df[investment_asset_share]
    back_testing_result = all_back_testing_df[['Total Investment Value', 'Investment PnL','Remain Cash', 'Total Value', 'Investment To Date. Return(%)']]
    
    return investment_asset_result, investment_share_value, back_testing_result


back_testing_df = Monthly_Rebalance(Stock_DF,price_column_name,investment_weights,initial_investment)
investment_asset_result, investment_share_value, back_testing_result = Adjust_Backtesting_Result(back_testing_df,price_column_name,investment_weights,initial_investment)

investment_asset_result

,AAPL,AMZN,MSFT,AAPL return,AMZN return,MSFT return
Date,,,,,,
2022-10-26,149.350006,115.660004,231.320007,NaN,NaN,NaN
2022-10-27,144.800003,110.959999,226.750000,-0.030465,-0.040636,-0.019756
2022-10-28,155.740005,103.410004,235.869995,0.075553,-0.068042,0.040220
2022-10-31,153.339996,102.440002,232.130005,-0.015410,-0.009380,-0.015856
2022-11-01,150.649994,96.790001,228.169998,-0.017543,-0.055154,-0.017059
2022-11-02,145.029999,92.120003,220.100006,-0.037305,-0.048249,-0.035368
2022-11-03,138.880005,89.300003,214.250000,-0.042405,-0.030612,-0.026579


In [9]:
investment_share_value

,AAPL #Share,AMZN #Share,MSFT #Share,AAPL Investment Value,AMZN Investment Value,MSFT Investment Value,Total Investment Value,Remain Cash,Total Value
Date,,,,,,,,,
2022-10-26,4,0,2,597.400024,0.000000,462.640015,1060.040039,239.959961,1300.000000
2022-10-27,4,0,2,579.200012,0.000000,453.500000,1032.700012,239.959961,1272.659973
2022-10-28,4,0,2,622.960022,0.000000,471.739990,1094.700012,239.959961,1334.659973
2022-10-31,4,0,2,613.359985,0.000000,464.260010,1077.619995,239.959961,1317.579956
2022-11-01,1,9,0,150.649994,871.110008,0.000000,1021.760002,351.679947,1373.439949
2022-11-02,1,9,0,145.029999,829.080025,0.000000,974.110023,351.679947,1325.789970
2022-11-03,1,9,0,138.880005,803.700027,0.000000,942.580032,351.679947,1294.259979


In [10]:
back_testing_result

,Total Investment Value,Investment PnL,Remain Cash,Total Value,Investment To Date. Return(%)
Date,,,,,
2022-10-26,1060.040039,0.000000,239.959961,1300.000000,1.000000
2022-10-27,1032.700012,-27.340027,239.959961,1272.659973,0.974208
2022-10-28,1094.700012,34.659973,239.959961,1334.659973,1.032697
2022-10-31,1077.619995,17.579956,239.959961,1317.579956,1.016584
2022-11-01,1021.760002,-38.280037,351.679947,1373.439949,0.963888
2022-11-02,974.110023,-85.930016,351.679947,1325.789970,0.918937
2022-11-03,942.580032,-117.460007,351.679947,1294.259979,0.889193
